# A0.4 Damped vibration - batch processing
by H cyan

huangzw29@mail2.sysu.edu.cn

2021.4.23

In [280]:
import glob,os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.interpolate import Rbf
from scipy.optimize import curve_fit
from scipy import fftpack

In [281]:
dfLIST = []
path = r'Steel_rule_data'
file = glob.glob(os.path.join(path, 'Raw Data**.csv'))
for f in file:
    dfLIST.append(pd.read_csv(f))
print(dfLIST) 

[     Time (s)  Linear Acceleration x (m/s^2)  Linear Acceleration y (m/s^2)  \
0    0.000000                      -0.040283                       0.371871   
1    0.019322                      -0.259062                       0.172190   
2    0.038634                      -0.125829                       0.151911   
3    0.057972                      -0.234843                       0.178468   
4    0.077279                      -0.220395                       0.088846   
..        ...                            ...                            ...   
424  8.193621                      -0.476855                       0.141034   
425  8.212940                      -0.523734                       0.156299   
426  8.232258                      -0.254575                      -0.243614   
427  8.251601                       0.361337                       0.072641   
428  8.270936                       0.314211                      -0.080735   

     Linear Acceleration z (m/s^2)  Absolute accel

## Indirect method

In [282]:
def Indirectmethod(df, idx):

    # Processing
    df.drop(df.head(100).index, inplace=True)
    df['Time (s)'] = df['Time (s)'] - df.iloc[0,0]
    
    # Original value
    t_origin = np.array(df['Time (s)'])
    a_origin = np.array(df['Linear Acceleration z (m/s^2)'])


    # Interpolation
    t_itp = np.linspace(t_origin.min(), t_origin.max(), 10000)
    func_itp = Rbf(t_origin, a_origin)
    # func_itp = UnivariateSpline(t_origin, a_origin)
    a_itp = func_itp(t_itp)


    # Caculate T
    zeropoint_rough=[]

    for i in a_itp:
        if i >= -0.2 and i <= 0.2:
            zeropoint_rough.append(t_itp[a_itp.tolist().index(i)])

    delta_t = np.diff(np.array(zeropoint_rough))
    delta_t = np.array(list(filter(lambda x: x >= 0.01, delta_t)))

    t_mean = np.mean(delta_t)
    t_sigma = np.std(delta_t, ddof = 1)

    delta_t = np.array(list(filter(lambda x: x <= (t_mean + 3*t_sigma) and x >= (t_mean - 3*t_sigma), delta_t)))

    t = 2*np.mean(delta_t)
    omega = 2*np.pi/t

    # Caculate the damped exponential
    ha = fftpack.hilbert(a_itp)
    a_env = np.sqrt(a_itp**2 + ha**2)

    def env(t, a, beta):
        return a*np.exp(-beta*t)
    
    popt, pcov = curve_fit(env, t_itp, a_env)
    avals = env(t_itp, popt[0], popt[1])

    a = popt[0]
    beta = popt[1]
    

    # Fit
    fit = a * np.exp(-beta*t_itp) * np.cos(omega*t_itp)


    # Plot
    fig, a =  plt.subplots(2,2,figsize=(20, 10))


    a[0][0].plot(t_origin, a_origin, linewidth=0.5, color='lightseagreen', label='Original curve')

    a[0][0].set_xlabel('time t/s')
    a[0][0].set_ylabel('Linear Acceleration a/(m/s^2)')
    a[0][0].set_title('(a) Original curve', y=-0.3)
    a[0][0].legend()


    a[0][1].plot(t_origin, a_origin, linewidth=0.5, color='lightseagreen', label='Original curve')
    a[0][1].plot(t_itp, avals, color='lightcoral', label='Envelope')

    a[0][1].set_xlabel('time t/s')
    a[0][1].set_ylabel('Linear Acceleration a/(m/s^2)')
    a[0][1].set_title('(b) Envelope', y=-0.3)
    a[0][1].legend()


    a[1][0].plot(t_itp, fit, color='deeppink', linewidth=0.5, label='Fitting curve')

    a[1][0].set_xlabel('time t/s')
    a[1][0].set_ylabel('Linear Acceleration a/(m/s^2)')
    a[1][0].set_title('(c) Fitting curve', y=-0.3)
    a[1][0].legend()


    a[1][1].plot(t_itp, fit, color='deeppink', linewidth=0.5, label='Fitting curve')
    a[1][1].plot(t_origin, a_origin, linewidth=0.5, color='lightseagreen', label='Original curve')
    a[1][1].plot(t_itp, avals, color='lightcoral', label='Envelope')

    a[1][1].set_xlabel('time t/s')
    a[1][1].set_ylabel('Linear Acceleration a/(m/s^2)')
    a[1][1].set_title('(d) Composite figure', y=-0.3)
    a[1][1].legend()


    plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.2, hspace=0.5)
    length = [130, 140, 160, 170, 180, 190]
    plt.suptitle('Fig.{}.1 Damped vibration of the steel rule (Indirect Fitting). L={}mm'.format(idx+1, length[idx]), y=0.01, fontsize=16)

    # Output
    plt.savefig(r'Steel_rule_fig_indirect\fig.{}.1.png'.format(idx+1), bbox_inches = 'tight')
    plt.clf()
    return (t, beta)



In [283]:
Tlist_ind = [] 
Betalist_ind = []
for idx in range(len(dfLIST)):
    df = dfLIST[idx]
    t, beta =Indirectmethod(df, idx)
    Tlist_ind.append(t) 
    Betalist_ind.append(beta)


<Figure size 1440x720 with 0 Axes>

<Figure size 1440x720 with 0 Axes>

<Figure size 1440x720 with 0 Axes>

<Figure size 1440x720 with 0 Axes>

<Figure size 1440x720 with 0 Axes>

<Figure size 1440x720 with 0 Axes>

In [284]:
print(Tlist_ind, Betalist_ind)

[0.1263569773312984, 0.13124330139920934, 0.16196049632721551, 0.17493381456717985, 0.19337115087787934, 0.21297310038751258] [0.3187085188730969, 0.27698589852746813, 0.1935851502612191, 0.1824842082428138, 0.1568400685974588, 0.14457378958316675]


In [285]:
length = np.array([130, 140, 160, 170, 180, 190])
ind = pd.DataFrame([length, np.array(Tlist_ind), np.array(Betalist_ind)], index=['L', 'T', 'beta'])
ind.loc['F'] = 1/ind.loc['T']
ind = ind.reindex(index=(['L', 'T', 'F', 'beta']))
print(ind)

               0           1           2           3           4           5
L     130.000000  140.000000  160.000000  170.000000  180.000000  190.000000
T       0.126357    0.131243    0.161960    0.174934    0.193371    0.212973
F       7.914086    7.619436    6.174345    5.716448    5.171402    4.695429
beta    0.318709    0.276986    0.193585    0.182484    0.156840    0.144574


In [286]:
writer = pd.ExcelWriter('Steel_rule_vibration_indirect.xlsx')
ind.to_excel(writer)
writer.save()

## Direct method

In [287]:
def Directmethod(df, idx):

    # Processing
    df.drop(df.head(100).index, inplace=True)
    df['Time (s)'] = df['Time (s)'] - df.iloc[0,0]
    
    # Original value
    t_origin = np.array(df['Time (s)'])
    a_origin = np.array(df['Linear Acceleration z (m/s^2)'])

    # Fitting
    def a_t(t, A1, bt, omg, A4, A5):
        return A1 * (omg**2) * np.exp(-bt*t) * np.cos(omg*t + A4) + A5
    param_bounds=([-10, 0, 20, -10, -10], [10, 0.5, 50, 10, 10])
    ## key: the max value of the omega. When selecting differnt values(40,50,60), some may success while others may fail.
    popt, pcov = curve_fit(a_t, t_origin, a_origin, bounds = param_bounds)
    t_dense = np.linspace(t_origin.min(), t_origin.max(), 10000)

    avals = a_t(t_dense, popt[0], popt[1], popt[2], popt[3], popt[4])
    Omega = popt[2]
    beta = popt[1]
    t = 2*np.pi/Omega

    # Plot
    fig, a =  plt.subplots(3,1,figsize=(10, 20))

    a[0].plot(t_origin, a_origin, linewidth=0.5, color='lightseagreen', label='Original curve')

    a[0].set_xlabel('time t/s')
    a[0].set_ylabel('Linear Acceleration a/(m/s^2)')
    a[0].set_title('(a) Original curve', y=-0.3)
    a[0].legend()


    a[1].plot(t_dense, avals, color='deeppink', linewidth=0.5, label='Fitting curve')

    a[1].set_xlabel('time t/s')
    a[1].set_ylabel('Linear Acceleration a/(m/s^2)')
    a[1].set_title('(b) Fitting curve', y=-0.3)
    a[1].legend()


    a[2].plot(t_dense, avals, color='deeppink', linewidth=0.5, label='Fitting curve')
    a[2].plot(t_origin, a_origin, linewidth=0.5, color='lightseagreen', label='Original curve')


    a[2].set_xlabel('time t/s')
    a[2].set_ylabel('Linear Acceleration a/(m/s^2)')
    a[2].set_title('(c) Composite figure', y=-0.3)
    a[2].legend()

    plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.2, hspace=0.5)
    length = [130, 140, 160, 170, 180, 190]
    plt.suptitle('Fig.{}.2 Damped vibration of the steel rule (Direct Fitting). L={}mm'.format(idx+1, length[idx]), y=0.05, fontsize=16)

    # Output
    plt.savefig(r'Steel_rule_fig_direct\fig.{}.2.png'.format(idx+1), bbox_inches = 'tight')
    plt.clf()
    return (t, beta)


In [288]:
Tlist_dir = [] 
Betalist_dir = []
for idx in range(len(dfLIST)):
    df = dfLIST[idx]
    t, beta =Directmethod(df, idx)
    Tlist_dir.append(t) 
    Betalist_dir.append(beta)

<Figure size 720x1440 with 0 Axes>

<Figure size 720x1440 with 0 Axes>

<Figure size 720x1440 with 0 Axes>

<Figure size 720x1440 with 0 Axes>

<Figure size 720x1440 with 0 Axes>

<Figure size 720x1440 with 0 Axes>

In [289]:
print(Tlist_dir, Betalist_dir)


[0.18079502349934165, 0.17580200290091916, 0.1690325199098668, 0.18288349416041613, 0.20140143874198915, 0.21920376009280038] [0.4999999999996496, 0.49999999999999994, 0.17572701086760562, 0.17487261157807074, 0.15111664645368567, 0.14082331137410758]


In [290]:
length = np.array([130, 140, 160, 170, 180, 190])
dir = pd.DataFrame([length, np.array(Tlist_dir), np.array(Betalist_dir)], index=['L', 'T', 'beta'])
dir.loc['F'] = 1/dir.loc['T']
dir = dir.reindex(index=(['L', 'T', 'F', 'beta']))
print(dir)

               0           1           2           3           4           5
L     130.000000  140.000000  160.000000  170.000000  180.000000  190.000000
T       0.180795    0.175802    0.169033    0.182883    0.201401    0.219204
F       5.531126    5.688217    5.916021    5.467962    4.965208    4.561966
beta    0.500000    0.500000    0.175727    0.174873    0.151117    0.140823


In [291]:
writer = pd.ExcelWriter('Steel_rule_vibration_direct.xlsx')
dir.to_excel(writer)
writer.save()